In [17]:
import cv2
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model, layers
from random import shuffle
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow import keras

In [11]:
NUM_FRAME = 10
IMG_SIZE = 100

LR = 0.0001
VIOLENCE_DIR = './Hockey_fights/Violence/'
NON_VIOLENCE_DIR = './Hockey_fights/Non-violence/'


Extracting Dataframes

In [11]:
count = 0
def extract_frames(folder):
    count = 0
    for files in os.listdir(folder):
        cap = cv2.VideoCapture(os.path.join(folder, files))
        sucess, image = cap.read()
        sucess = True
        while sucess:
            sucess, image = cap.read()
            cv2.imwrite("./Hockey_fights/Dataframes/"+ str(count) + ".jpg", image)

            if cv2.waitKey(10) == 27:
                break
            count += 1






In [5]:
count = 0
for files in os.listdir(VIOLENCE_DIR):
    cap = cv2.VideoCapture(os.path.join(VIOLENCE_DIR, files))
    sucess, image = cap.read()
    #print(sucess)
    sucess = True
    while sucess:
        sucess, image = cap.read()
        if not sucess:
            break 
        cv2.imwrite("./Hockey_fights/Dataframes/"+ str(count) + ".jpg", image)

        if cv2.waitKey(10) == 27:
            break
        count += 1

KeyboardInterrupt: 

In [20]:
print(count)

20057


In [1]:
for files in os.listdir(NON_VIOLENCE_DIR):
    cap = cv2.VideoCapture(os.path.join(NON_VIOLENCE_DIR, files))
    sucess, image = cap.read()
    #print(sucess)
    sucess = True
    while sucess:
        sucess, image = cap.read()
        if not sucess:
            break 
        cv2.imwrite("./Hockey_fights/Dataframes/"+ str(count) + ".jpg", image)

        if cv2.waitKey(10) == 27:
            break
        count += 1

NameError: name 'NON_VIOLENCE_DIR' is not defined

In [22]:
print(count)

40056


In [12]:
def create_dataset():
    dataset = []
    image = []
    limit = 0
    c = 0

    for file in tqdm(os.listdir('./Hockey_fights/Dataframes/')):
        path = os.path.join('./Hockey_fights/Dataframes/', file)
        img = cv2.resize(cv2.imread(path), (IMG_SIZE, IMG_SIZE))
        image.append(np.array(img))
        limit += 1
        c += 1
        if c == NUM_FRAME:
            c = 0
            if limit < 20057:
                dataset.append([image, np.array([1, 0])])
            elif limit >= 20056:
                dataset.append([image, np.array([0, 1])])
            image = []
    
    shuffle(dataset)
    np.save('dataset.npy', dataset)
    return dataset



In [13]:
data = create_dataset()

100%|██████████| 40056/40056 [00:47<00:00, 843.81it/s]
C:\Users\User\AppData\Local\Programs\Python\Python37\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [14]:
train = data[:-400]
test = data[-400:]

Creating the Model

In [194]:
from keras import backend as K
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, TimeDistributed,Dropout, Activation, Flatten,Conv2D, MaxPooling2D,LSTM,Bidirectional, Reshape
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import Input

inputs = Input(shape=[10, IMG_SIZE, IMG_SIZE, 3], name= "input")
convrnn = Conv2D(64, (3, 3), activation='relu', padding = 'same')(inputs)
convrnn = Conv2D(64, (3, 3), activation='relu')(convrnn)
print(convrnn[0][0].shape)
convrnn = MaxPooling2D((2,2))(convrnn[0])
print(convrnn.shape)
convrnn = Conv2D(64, (3, 3), activation='relu')(convrnn)
print(convrnn.shape)
convrnn = MaxPooling2D((2,2))(convrnn)
print(convrnn.shape)
convrnn = Conv2D(64, (3, 3), activation='relu')(convrnn)
convrnn = MaxPooling2D((2,2))(convrnn)
print(convrnn.shape)
convrnn = Flatten()(convrnn)
print(convrnn.shape)

convrnn = tf.reshape(convrnn, (1, 10, 6400))
# print(convrnn.shape)

lstm_fw = LSTM(units=32)
lstm_bw = LSTM(units=32, go_backwards=True)
bi_lstm = Bidirectional(lstm_fw, backward_layer=lstm_bw)(convrnn)

convrnn = Dense(64, activation= 'relu')(bi_lstm)
convrnn = Dense(32, activation= 'relu')(convrnn)
output = Dense(1, activation='sigmoid')(convrnn)

model = tf.keras.Model(inputs = inputs, outputs = output)
model.summary()

(98, 98, 64)
(10, 49, 49, 64)
(10, 47, 47, 64)
(10, 23, 23, 64)
(10, 10, 10, 64)
(10, 6400)
Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 10, 100, 100, 3)] 0         
_________________________________________________________________
conv2d_177 (Conv2D)          (None, 10, 100, 100, 64)  1792      
_________________________________________________________________
conv2d_178 (Conv2D)          (None, 10, 98, 98, 64)    36928     
_________________________________________________________________
tf.__operators__.getitem_241 (10, 98, 98, 64)          0         
_________________________________________________________________
max_pooling2d_177 (MaxPoolin (10, 49, 49, 64)          0         
_________________________________________________________________
conv2d_179 (Conv2D)          (10, 47, 47, 64)          36928     
________________________________

In [17]:
len(train[0][1])

2

In [18]:
from tensorflow.keras.layers import LSTM, Bidirectional, Conv2D, Dense, Flatten, MaxPooling2D, TimeDistributed, Reshape, MaxPooling3D
from tensorflow.keras.models import Sequential

cnn = Sequential()
cnn.add(Conv2D(64, (3, 3), activation='relu', input_shape=(10, IMG_SIZE, IMG_SIZE, 3), padding="same"))
cnn.add(Conv2D(64, (3, 3), activation='relu'))
cnn.add(MaxPooling3D((1, 2,2)))
cnn.add(Conv2D(64, (3, 3), activation='relu'))
cnn.add(MaxPooling3D((1, 2,2)))
cnn.add(Conv2D(64, (3, 3), activation='relu'))
cnn.add(MaxPooling3D((1, 2,2)))
#cnn.add(Flatten())
cnn.add(Reshape((10, 6400)))

#cnn.summary()

lstm_fw = LSTM(units=32)
lstm_bw = LSTM(units=32, go_backwards = True)
model = Sequential()
#model.add(TimeDistributed(cnn, input_shape=(1, 10, 10, 64)))
model.add(Bidirectional(lstm_fw, backward_layer = lstm_bw))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

cnn.add(Bidirectional(lstm_fw, backward_layer = lstm_bw))
cnn.add(Dense(64, activation='relu'))
cnn.add(Dense(32, activation='relu'))
cnn.add(Dense(2, activation='sigmoid'))

cnn.summary()
opt = keras.optimizers.Adam(learning_rate=LR)
cnn.compile(loss='categorical_crossentropy',optimizer='SGD',metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 10, 100, 100, 64)  1792      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 10, 98, 98, 64)    36928     
_________________________________________________________________
max_pooling3d_3 (MaxPooling3 (None, 10, 49, 49, 64)    0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 10, 47, 47, 64)    36928     
_________________________________________________________________
max_pooling3d_4 (MaxPooling3 (None, 10, 23, 23, 64)    0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 10, 21, 21, 64)    36928     
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 10, 10, 10, 64)   

In [3]:
data = np.load('dataset.npy', allow_pickle=True)

In [5]:
train, test = train_test_split(data, train_size= 0.9, shuffle=True)


In [9]:
train[0][1]

array([0, 1])

In [12]:
X = np.array([i[0] for i in train]).reshape(-1, 10, IMG_SIZE, IMG_SIZE, 3)
Y = np.array([i[1] for i in train])

x_valid = np.array([i[0] for i in test]).reshape(-1, 10, IMG_SIZE, IMG_SIZE, 3)
y_valid = np.array([i[1] for i in test])

In [13]:

X = X.astype('float32')/255
x_valid = x_valid.astype('float32')/255
#Y = Y.astype('float32').reshape((-1, 1))
#y_valid = y_valid.astype('float32').reshape((-1, 1))



In [14]:
print(Y.shape)
print(y_valid.shape)
print(X.shape)
print(x_valid.shape)

print(y_valid[7])

(3604, 2)
(401, 2)
(3604, 10, 100, 100, 3)
(401, 10, 100, 100, 3)
[0 1]


In [192]:
x_try = X[:20]
y_try = Y[:20]
print(len(x_try), len(y_try))

20 20


In [22]:
cnn.fit(X,Y,epochs=10,validation_data=(x_valid,y_valid),batch_size=16, verbose=1)

Epoch 1/10
226/226 [==============================] - 76s 337ms/step - loss: 0.0823 - accuracy: 0.9673 - val_loss: 0.3053 - val_accuracy: 0.8953
Epoch 2/10
226/226 [==============================] - 76s 337ms/step - loss: 0.1207 - accuracy: 0.9600 - val_loss: 0.1414 - val_accuracy: 0.9401
Epoch 3/10
226/226 [==============================] - 77s 339ms/step - loss: 0.0800 - accuracy: 0.9714 - val_loss: 0.1408 - val_accuracy: 0.9451
Epoch 4/10
226/226 [==============================] - 76s 338ms/step - loss: 0.0415 - accuracy: 0.9881 - val_loss: 0.1304 - val_accuracy: 0.9501
Epoch 5/10
226/226 [==============================] - 76s 338ms/step - loss: 0.0378 - accuracy: 0.9881 - val_loss: 0.1232 - val_accuracy: 0.9476
Epoch 6/10
226/226 [==============================] - 77s 339ms/step - loss: 0.0190 - accuracy: 0.9950 - val_loss: 0.1045 - val_accuracy: 0.9501
Epoch 7/10
226/226 [==============================] - 76s 338ms/step - loss: 0.0341 - accuracy: 0.9895 - val_loss: 1.7414 - val_ac

In [24]:
cnn.save('CNN-BiLSTM.h5', overwrite=True, include_optimizer=True)